## Ноут-шаблон под обучение моделей
> **Важно: обязательно фиксируем рандом (`seed=42`) для всех библиотек и методов, где под капотом подразумевается рандом!** 

> **Почему это важно:** для гарантии консистентности данных при сравнении разных моделей и решений.


In [2]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from gensim.models import FastText


from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC


import random
import ast

# фиксирую то, что импортнул
random.seed(42)
np.random.seed(42)

In [ ]:
# Скачиваем необходимые ресурсы (выполнить один раз)
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger_eng') <-- Новый пакет для POS-тегов

### Импорт датасета и сплит на выборки

In [3]:
# Загружаем датасет
df = pd.read_csv("customer_support_dataset_generated.csv")
df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [5]:
df.loc[28120]

flags                                                        BLC
instruction    I want to create an account and also see my de...
category                                 ['ACCOUNT', 'DELIVERY']
intent                    ['create_account', 'delivery_options']
response       Certainly! To create a new account, please hea...
Name: 28120, dtype: object

### Предобработка

In [6]:
# Обязательный блок: исправляем дублящиеся категории и интенты
def process_category_unique(value):
    if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
        try:
            categories = ast.literal_eval(value)
            unique_categories = list(set(categories))
            # Если осталась одна категория - возвращаем как строку
            if len(unique_categories) == 1:
                return unique_categories[0]
            else:
                return ', '.join(sorted(unique_categories))
        except:
            return value
    else:
        return value

    
    
df['category'] = df['category'].apply(process_category_unique).astype(str)
df['intent'] = df['intent'].apply(process_category_unique).astype(str)

In [7]:
df.loc[28120]

flags                                                        BLC
instruction    I want to create an account and also see my de...
category                                       ACCOUNT, DELIVERY
intent                          create_account, delivery_options
response       Certainly! To create a new account, please hea...
Name: 28120, dtype: object

**Дальше я делаю подготовку текстовок обращений**

В целом, этот блок на ваше усмотрение можно кастомить, если я что-то забыл учесть, или у вас появилась классная идея, ~~или я где-то жестоко не прав и ошибся~~, но базовый минимум здесь - дропнуть английские стоп-слова и пунктуацию, привести слова к леммам

In [ ]:
import re
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


class TextPreprocessor:
    def __init__(self, language='english', custom_stopwords=None):
        
        self.stop_words = set(stopwords.words(language))
        
        # Удаляем отрицания из стоп-слов, чтобы не терять смысл
        negations = {'not', 'no', 'never', "don't", "isn't", "wasn't", "couldn't", "aren't"}
        self.stop_words.difference_update(negations)
        
        # Для добавления кастомных стоп-слов
        if custom_stopwords:
            self.stop_words.update(custom_stopwords)
        
        self.lemmatizer = WordNetLemmatizer()
        
        # Шаблоны для удаления
        self.patterns_to_remove = [
            r'{{.*?}}', r'\[.*?\]', r'<.*?>', r'http\S+', r'@\w+', r'#\w+'
        ]
    
    def get_wordnet_pos(self, word):
        """
        Функция определяет часть речи (POS-тег)
        для более качественной лемматизации.
        """
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN) # по умолчанию считаем существительным

    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        
        text = text.lower()
        
        for pattern in self.patterns_to_remove:
            text = re.sub(pattern, '', text)
        
        # Заменяем жесткое удаление на более гибкое.
        # Удаляем все, что не является буквой, цифрой или пробелом.
        # Это сохраняет номера заказов и артикулы.
        text = re.sub(r'[^a-z0-9\s]', '', text)
        
        tokens = word_tokenize(text)
        
        filtered_tokens = []
        for token in tokens:
            if len(token) > 1 and token not in self.stop_words:
                
                # Лемматизация теперь использует POS-теги (Part Of Speech)
                pos_tag = self.get_wordnet_pos(token)
                lemma = self.lemmatizer.lemmatize(token, pos_tag)
                
                filtered_tokens.append(lemma)
        
        return " ".join(filtered_tokens)

In [12]:
preprocessor = TextPreprocessor(language='english', custom_stopwords='fucking')

df['clean_instruction'] = df['instruction'].apply(preprocessor.clean_text)

In [10]:
preprocessor.stop_words # для понимания, что там лежит 

# no, not -> вероятно не нужно исключать, подумать
# i do **not** want to cancel order, i want to change shipping address

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'isn',
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'nor',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out'

In [11]:
df[['instruction', 'clean_instruction']].sample(25)

,instruction,clean_instruction
4630,checking invoice,check invoice
15406,I call to download bill #00108,call download bill
13070,help me see when will myarticle arrive,help see myarticle arrive
28738,Is there a delivery period for this item?,delivery period item
19525,can uhelp me earn several products,uhelp earn several product
33072,I need to switch to a different account.,need switch different account
26663,I have to know if there is anything wrong with...,know anything wrong rebate
20496,I have got to retrieve the PIN of my account,get retrieve pin account
816,assistance to cancel purchase {{Order Number}},assistance cancel purchase
33378,How do I sign up for a new service account?,sign new service account


### Сплит выборки
Разбиваем на:
- `train` - на нём обучаем модели
- `validation` - на нём "тюним" модели, настраимаем гиперпараметры (когда GridSearch делаем, к примеру)
- `test` - финальная выборка для оценки качества  Тестовый набор используется **ТОЛЬКО ОДИН РАЗ** в самом конце! При обучении и подборе параметров модель ничего не должна знать из этого датасета.

Предлагаю бить в пропорции **70/15/15**

- train
- validation для экспериментов
- test откладываем и забываем - его доразбить для имитации сервиса

In [13]:
def train_val_test_split(X, 
                         y, 
                         val_size=0.15, 
                         test_size=0.15, 
                         random_state=42 # !!!
                         ):
    """
    Разделяет данные на train, validation и test наборы
    """

    # Сначала отделяем test
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, 
        y, 
        test_size=test_size, 
        random_state=random_state, 
        shuffle=True
        # stratify=y  # есть классы всего лишь с 1 примером, поэтому просто шафлим
    )
    
    # Затем разделяем на train и validation
    relative_val_size = val_size / (1 - test_size)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, 
        y_train_val, 
        test_size=relative_val_size, 
        random_state=random_state, 
        shuffle=True
        # stratify=y_train_val  # есть классы всего лишь с 1 примером, поэтому просто шафлим
    )
    
    return X_train, X_val, X_test, y_train, y_val, y_test



# Получаем выборки
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(
    X=df.drop(['intent'], axis=1), 
    y=df[['intent']]
)

In [14]:
df.shape, X_train.shape, X_val.shape, X_test.shape

((40366, 6), (28256, 5), (6055, 5), (6055, 5))

### Векторизация
Пробуем следующие методы:
- Bag of Words (мешок слов)
- TF-IDF
- Word2Vec
- FastText

Статья, которой можно вдохновиться: https://habr.com/ru/articles/778048/

#### Bag of Words

In [16]:
# Bag of Words

vec_bag = CountVectorizer()
X_train_bag = vec_bag.fit_transform(X_train['clean_instruction'])
X_val_bag = vec_bag.transform(X_val['clean_instruction']) 

# Результаты
print(pd.DataFrame(X_train_bag.toarray(), columns=vec_bag.get_feature_names_out()))

       00004587345current  00123842current  10  10101  101010  11111  11223  \
0                       0                0   0      0       0      0      0   
1                       0                0   0      0       0      0      0   
2                       0                0   0      0       0      0      0   
3                       0                0   0      0       0      0      0   
4                       0                0   0      0       0      0      0   
...                   ...              ...  ..    ...     ...    ...    ...   
28251                   0                0   0      0       0      0      0   
28252                   0                0   0      0       0      0      0   
28253                   0                0   0      0       0      0      0   
28254                   0                0   0      0       0      0      0   
28255                   0                0   0      0       0      0      0   

       112233  112233445  113542617735902current  .

#### TF-IDF

In [17]:
# TF-IDF

vec_tfidf = TfidfVectorizer()
X_train_tfidf = vec_tfidf.fit_transform(X_train['clean_instruction'])
X_val_tfidf = vec_tfidf.transform(X_val['clean_instruction'])

# Результаты
print(pd.DataFrame(X_train_tfidf.toarray(), columns=vec_tfidf.get_feature_names_out()))

       00004587345current  00123842current   10  10101  101010  11111  11223  \
0                     0.0              0.0  0.0    0.0     0.0    0.0    0.0   
1                     0.0              0.0  0.0    0.0     0.0    0.0    0.0   
2                     0.0              0.0  0.0    0.0     0.0    0.0    0.0   
3                     0.0              0.0  0.0    0.0     0.0    0.0    0.0   
4                     0.0              0.0  0.0    0.0     0.0    0.0    0.0   
...                   ...              ...  ...    ...     ...    ...    ...   
28251                 0.0              0.0  0.0    0.0     0.0    0.0    0.0   
28252                 0.0              0.0  0.0    0.0     0.0    0.0    0.0   
28253                 0.0              0.0  0.0    0.0     0.0    0.0    0.0   
28254                 0.0              0.0  0.0    0.0     0.0    0.0    0.0   
28255                 0.0              0.0  0.0    0.0     0.0    0.0    0.0   

       112233  112233445  1135426177359

#### Word2Vec

In [18]:
# Word2Vec

X_train_tokenized_docs = [word_tokenize(doc) for doc in X_train['clean_instruction']]
X_val_tokenized_docs = [word_tokenize(doc) for doc in X_val['clean_instruction']]

word2vec_model = Word2Vec(
    sentences=X_train_tokenized_docs,
    vector_size=100,     # размер вектора
    window=5,            # размер окна контекста
    min_count=1,         # минимальная частота слова
    workers=1,           # количество потоков, 1 - иначе рандомит
    sg=1                 # 1 для skip-gram, 0 для CBOW
)

word2vec_model.train(X_train_tokenized_docs, total_examples=len(X_train_tokenized_docs), epochs=10)

# взять предобученный w2v

(743196, 1185370)

In [ ]:
import gensim.downloader as api

# Скачиваем предобученную модель (Glove Twitter, размер вектора 100)
pretrained_model = api.load("glove-twitter-100")

# Токенизация (превращаем предложения в списки слов)
X_train_tokenized_docs = [word_tokenize(doc) for doc in X_train['clean_instruction']]
X_val_tokenized_docs = [word_tokenize(doc) for doc in X_val['clean_instruction']]

# Функция для усреднения векторов (немного адаптированная под предобученную модель)
def document_vector(model, doc):
    # Фильтруем слова: оставляем только те, что есть в словаре модели
    # В предобученной модели мы обращаемся к словарю через model.key_to_index (или просто проверяем in model)
    words = [word for word in doc if word in model.key_to_index]
    
    if len(words) == 0:
        # Если в предложении нет ни одного знакомого слова (или оно пустое),
        # возвращаем вектор из нулей, чтобы код не упал с ошибкой
        return np.zeros(model.vector_size)
    
    # model[words] возвращает массив векторов для всех найденных слов.
    # np.mean усредняет их в один вектор
    return np.mean(model[words], axis=0)

# 4. Применяем функцию ко всем документам
X_train_w2v = np.array([document_vector(pretrained_model, doc) for doc in X_train_tokenized_docs])
X_val_w2v = np.array([document_vector(pretrained_model, doc) for doc in X_val_tokenized_docs])

print(f"Размерность обучающей матрицы: {X_train_w2v.shape}")

[==================================================] 100.0% 387.1/387.1MB downloaded
Размерность обучающей матрицы: (28256, 100)


In [21]:
# Получение векторов слов
print("Вектор для слова 'help':")
print(word2vec_model.wv['help'])

# Похожие слова
print("\nПохожие слова на 'help':")
print(word2vec_model.wv.most_similar('help', topn=10))

Вектор для слова 'help':
[-2.97602229e-02  2.87921399e-01  2.08012775e-01  2.61957318e-01
 -2.13927090e-01 -7.44856298e-01  4.54741389e-01  1.29259288e-01
 -5.81863642e-01 -7.37532377e-02  4.36909497e-01 -1.42000780e-01
  3.84372711e-01  3.71176720e-01  2.15179585e-02 -3.18511516e-01
  1.67894587e-01 -4.42687720e-01 -3.07508737e-01 -9.98989224e-01
  2.10191026e-01  1.51525095e-01  7.24495292e-01  3.41706306e-01
  7.15186298e-01 -7.58386254e-01 -2.88622286e-02 -1.53265476e-01
  5.16312383e-03 -6.02941573e-01  3.00668895e-01  9.05029029e-02
  9.36316475e-02 -4.50585395e-01 -1.07777260e-01  3.85010988e-01
  6.55538142e-01 -3.71116877e-01 -9.96763334e-02 -1.59316987e-01
 -2.55321175e-01 -5.00283957e-01  5.76937757e-02 -1.08016618e-01
 -2.20560171e-02  1.24073669e-01 -6.93714857e-01  3.58207494e-01
 -2.47616395e-01  6.71634376e-01  1.49149656e-01 -5.24880439e-02
 -9.14265573e-01  1.22338273e-01  2.42298841e-01 -1.14371881e-01
 -1.22924903e-02 -3.50149274e-01 -8.68295506e-02  3.42357039e-01


In [24]:
# Получение векторов слов
print("Вектор для слова 'help':")
print(pretrained_model['help'])

# Похожие слова
print("\nПохожие слова на 'help':")
print(pretrained_model.most_similar('help', topn=10))

Вектор для слова 'help':
[ 0.41755   0.68889  -0.46208   0.35115  -0.23694  -0.68691   1.1901
  0.23397   0.37881  -0.23358   0.33652  -0.039475 -4.3266    0.62525
 -0.80124  -0.37718  -0.13806  -0.72706  -0.20413  -0.73376   0.14369
 -0.049832 -0.79746   0.55831  -0.20469   0.12483   0.19367   0.99597
  0.38962   0.23743  -0.3175   -0.17651  -0.60194   0.52959   0.70239
 -0.40085   0.74437   0.08028   0.075221  0.58811  -0.55258   0.008635
  0.15862  -0.34466   0.38218   0.2284   -0.37994  -0.041245 -0.51875
  0.32053   0.12853  -0.59126   0.53532   0.010341  0.10907  -0.21717
 -0.59941   0.78575  -0.37995   0.2596   -0.18696   0.39263   0.50281
  0.23801   0.7866    0.035248 -0.22407   0.067952 -0.46081   0.21476
 -0.012484  0.36379   0.51241   0.58936  -0.21477  -0.24591  -0.1853
 -0.47781  -0.4952   -0.57234   1.1957    0.41207   0.19577  -0.5065
  0.19058  -0.33058  -0.74558  -0.40009   0.25671   0.54588   0.084593
 -0.42773   0.43142  -0.25585   0.059181 -0.45639  -0.1141    0.37

#### FastText

In [ ]:
# FastText

fasttext_model = FastText(
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=1
)

# Обучение модели
fasttext_model.build_vocab(X_train_tokenized_docs)
fasttext_model.train(
    X_train_tokenized_docs, 
    total_examples=len(X_train_tokenized_docs), 
    epochs=20
)

(743480, 1185370)

In [29]:
# FastText может работать с OOV (out-of-vocabulary) словами
print("Вектор для существующего слова 'help':")
print(fasttext_model.wv['help'])

# Вектор для несуществующего слова (FastText использует n-grams)
fake_word = 'looooooooooooooool'
print(f"\nВектор для OOV слова '{fake_word}':")
print(fasttext_model.wv[fake_word])

# Похожие слова
print("\nПохожие слова на 'help':")
print(fasttext_model.wv.most_similar('help', topn=10))
print(fasttext_model.wv.most_similar('modify'))
print(fasttext_model.wv.most_similar('omdify'))

Вектор для существующего слова 'help':
[-0.280823   -0.52019906  0.04496343  0.10337804 -0.23796153  0.0341261
 -0.19240457  0.5989376   0.3635937  -0.9477554   0.09990043 -0.35251018
 -0.10727239  0.28350145  0.5472687  -0.7253978   0.244883    0.07058468
 -0.3493659  -0.50163466  0.17107655 -0.08316001  0.27093768  0.15646604
 -0.06674644 -0.64147496  0.02320246  0.16206717 -0.5552184   0.14339775
 -0.82274586  0.17867315  0.77332747  0.13780138 -0.23408133 -0.15446204
 -0.535891   -0.6433006   0.43265593  0.6328216  -0.45555556 -0.32184964
 -0.2102307  -0.32122886 -0.47794172 -0.17973186  0.31489056  0.11421549
  0.04988303  0.4028192   0.63853204 -0.5740908   0.20030802 -0.02928838
  0.18040204 -0.05406512  0.05711975  0.02496478  0.5413362   0.2841979
  0.15843563  0.04333622 -0.17841671  0.06021926 -0.2715158   0.31578878
  0.24584891 -0.27004483  0.3954763   0.16044621 -0.05408617 -0.08031064
  0.24039108 -0.31412217  0.02891675 -0.19632743  0.35999453  0.8099027
 -0.09918266  0

In [33]:
def document_vector_fasttext(model, doc):
    # doc - это список слов ['i', 'need', 'omdify', 'address']
    
    # У FastText нет понятия "незнакомое слово", если оно состоит из букв,
    # которые он видел, поэтому берем все слова
    vectors = [model.wv[word] for word in doc]
    
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    
    return np.mean(vectors, axis=0)


X_train_ft = np.array([document_vector_fasttext(fasttext_model, doc) for doc in X_train_tokenized_docs])
X_val_ft = np.array([document_vector_fasttext(fasttext_model, doc) for doc in X_val_tokenized_docs])
print(f"Длина вектора: {X_train_ft[0].shape}")

Длина вектора: (100,)


### Классификация интентов
**Модели**

Точно пробуем из линейных моделей SVM и логистическую регрессию, но в целом можно поэкспериментировать.

Для многоклассовой классификации - можно воспользоваться "One-vs-Rest" и "One-vs-One"

**Метрики** 

f1-score, precision и recall (micro, macro и weighted - всё имплементировано в sklearn'е)

Пара штрихов перед обучением

In [ ]:
# для multilabeled - делаем, чтобы всё хранилось в виде списков

y_train['intent'] = y_train['intent'].apply(lambda x: x.split(', ') if isinstance(x, str) else [x])
y_val['intent'] = y_val['intent'].apply(lambda x: x.split(', ') if isinstance(x, str) else [x])
y_test['intent'] = y_test['intent'].apply(lambda x: x.split(', ') if isinstance(x, str) else [x])

In [35]:
# бинарайзер - для мультилейбл классификатора
mlb = MultiLabelBinarizer()
y_train_bin = mlb.fit_transform(y_train['intent'])
y_val_bin = mlb.transform(y_val['intent'])
y_test_bin = mlb.transform(y_test['intent'])

y_train_bin

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [36]:
X_train_list = [X_train_bag.toarray(), X_train_tfidf.toarray(), X_train_w2v, X_train_ft]
X_val_list = [X_val_bag.toarray(), X_val_tfidf.toarray(), X_val_w2v, X_val_ft]

X_train_val_list = [[X_train_bag.toarray(), X_val_bag.toarray()], 
                    [X_train_tfidf.toarray(), X_val_tfidf.toarray()], 
                    [X_train_w2v, X_val_w2v], # w2v здесь предобученная модель
                    [X_train_ft, X_val_ft]
                   ]

# X_test_list = [X_test_bag.toarray(), X_test_tfidf.toarray(), X_test_w2v, X_test_ft]

#### LogisticRegression

In [ ]:
logreg_list = []
y_pred_list = []

for x_train_val in X_train_val_list:
    # MultiOutputClassifier - обёртка для multilabeles классификации
    # пока сделаем с дефолтными параметрами
    model = MultiOutputClassifier(LogisticRegression(random_state=42))
    model.fit(x_train_val[0], y_train_bin)

    # Предсказания
    y_pred = model.predict(x_train_val[1])
    
    # Сохранили модель и предикт
    logreg_list.append(model)
    y_pred_list.append(y_pred)

/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

In [38]:
logreg_list = []
y_pred_list = []

for x_train_val in X_train_val_list:
    # MultiOutputClassifier - обёртка для multilabel
    # поменяли solver
    model = MultiOutputClassifier(LogisticRegression(
    random_state=42, 
    max_iter=1000,         # даем больше времени на обучение
    class_weight='balanced', # чтобы не игнорировать редкие запросы
    solver='liblinear'   # с saga не вышло
    ))
    model.fit(x_train_val[0], y_train_bin)

    # Предсказания
    y_pred = model.predict(x_train_val[1])
    
    # Сохранили модель и предикт
    logreg_list.append(model)
    y_pred_list.append(y_pred)

In [39]:
vecs = ['BoW', 'TF-IDF', 'W2V', 'FT']
for i in range(4):
    print(vecs[i])
    print("Micro F1:", f1_score(y_val_bin, y_pred_list[i], average='micro'))
    print("Macro F1:", f1_score(y_val_bin, y_pred_list[i], average='macro')) 
    print("Weighted F1:", f1_score(y_val_bin, y_pred_list[i], average='weighted'))
    print(classification_report(y_val_bin, y_pred_list[i]))
    print('-'*10)

    
# модель предсказала все нули???
# стоит ли сделать , zero_division=0?

BoW
Micro F1: 0.9079699248120301
Macro F1: 0.9109034654898237
Weighted F1: 0.9113462918978267
              precision    recall  f1-score   support

           0       0.90      0.99      0.95       228
           1       0.73      0.99      0.84       183
           2       0.90      0.99      0.94       228
           3       0.98      1.00      0.99       226
           4       0.75      0.98      0.85       246
           5       0.95      1.00      0.97       231
           6       0.81      0.99      0.89       257
           7       0.94      0.95      0.95       222
           8       0.81      0.97      0.88       205
           9       0.90      0.99      0.94       254
          10       0.78      1.00      0.88       226
          11       0.94      1.00      0.97       263
          12       0.75      1.00      0.86       249
          13       0.86      0.97      0.91       216
          14       0.70      0.99      0.82       206
          15       0.69      0.97      0.

c:\Users\1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifi

In [44]:
# Берем векторизатор Bag of Words, который мы обучили на Train.
# используем только .transform()
print("Векторизация тестовой выборки...")
X_test_bag = vec_bag.transform(X_test['clean_instruction']).toarray()

# y_test_bin = mlb.transform(y_test['intent'])
print(f"Размер X_test: {X_test_bag.shape}")
print(f"Размер y_test: {y_test_bin.shape}")


# так как в цикле векторов ['BoW', 'TF-IDF'...] BoW шел первым:
model_bow = logreg_list[0]

print("Предсказание на тестовых данных...")
y_pred_test = model_bow.predict(X_test_bag)

print("-" * 40)
print("ИТОГОВЫЕ МЕТРИКИ НА TEST (Bag of Words)")
print("-" * 40)

print(f"Micro F1:    {f1_score(y_test_bin, y_pred_test, average='micro'):.4f}")
print(f"Macro F1:    {f1_score(y_test_bin, y_pred_test, average='macro'):.4f}")
print(f"Weighted F1: {f1_score(y_test_bin, y_pred_test, average='weighted'):.4f}")
print("\nПолный отчет:")
print(classification_report(y_test_bin, y_pred_test))

Векторизация тестовой выборки...
Размер X_test: (6055, 3277)
Размер y_test: (6055, 27)
Предсказание на тестовых данных...
----------------------------------------
ИТОГОВЫЕ МЕТРИКИ НА TEST (Bag of Words)
----------------------------------------
Micro F1:    0.9082
Macro F1:    0.9111
Weighted F1: 0.9113

Полный отчет:
              precision    recall  f1-score   support

           0       0.93      0.98      0.96       246
           1       0.76      0.98      0.86       228
           2       0.86      0.98      0.92       210
           3       0.98      0.99      0.99       258
           4       0.76      0.99      0.86       217
           5       0.94      1.00      0.97       248
           6       0.80      0.98      0.88       262
           7       0.94      0.94      0.94       193
           8       0.85      0.97      0.91       240
           9       0.90      0.98      0.94       249
          10       0.81      0.98      0.89       202
          11       0.91      1.0

c:\Users\1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
from sklearn.metrics import accuracy_score, jaccard_score
 
# 1. Exact Match Ratio (Идеальное совпадение)
# Показывает % тикетов, где модель не ошиблась ни в одной запятой
exact_match = accuracy_score(y_test_bin, y_pred_test)

# 2. Jaccard Score (samples average)
# Показывает средний % перекрытия правильных и предсказанных тегов
jaccard = jaccard_score(y_test_bin, y_pred_test, average='samples')

# 3. Кастомная метрика: "Хотя бы один верный" (At Least One Match)
def at_least_one_accuracy(y_true, y_pred):
    # Умножаем матрицы: если есть совпадение 1*1, будет > 0
    matches = (y_true * y_pred).sum(axis=1)
    # Считаем долю строк, где сумма > 0
    return (matches > 0).mean()

at_least_one = at_least_one_accuracy(y_test_bin, y_pred_test)

print(f"--- ДОПОЛНИТЕЛЬНЫЕ МЕТРИКИ (BoW) ---")
print(f"Exact Match Ratio (Полная автоматизация): {exact_match:.4f}")
print(f"Jaccard Score (Степень похожести):        {jaccard:.4f}")
print(f"At Least One Match (Полезное действие):   {at_least_one:.4f}")

--- ДОПОЛНИТЕЛЬНЫЕ МЕТРИКИ (BoW) ---
Exact Match Ratio (Полная автоматизация): 0.8296
Jaccard Score (Степень похожести):        0.9057
At Least One Match (Полезное действие):   0.9871


### LinearSVC

In [46]:
# LinearSVC

svc_list = []
y_svc_pred_list = []

for x_train_val in X_train_val_list:
    # MultiOutputClassifier - обёртка для multilabel классификации
    # поменяли solver
    model = MultiOutputClassifier(LinearSVC(
        random_state=42, 
        class_weight='balanced', 
        max_iter=2000
    ))
    model.fit(x_train_val[0], y_train_bin)

    # Предсказания
    y_pred = model.predict(x_train_val[1])
    
    # Сохранили модель и предикт
    svc_list.append(model)
    y_svc_pred_list.append(y_pred)

In [47]:
vecs = ['BoW', 'TF-IDF', 'W2V', 'FT']
for i in range(4):
    print(vecs[i])
    print("Micro F1:", f1_score(y_val_bin, y_svc_pred_list[i], average='micro'))
    print("Macro F1:", f1_score(y_val_bin, y_svc_pred_list[i], average='macro')) 
    print("Weighted F1:", f1_score(y_val_bin, y_svc_pred_list[i], average='weighted'))
    print(classification_report(y_val_bin, y_svc_pred_list[i]))
    print('-'*10)

BoW
Micro F1: 0.9317438055165965
Macro F1: 0.9336784976578885
Weighted F1: 0.9336853071175403
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       228
           1       0.88      0.95      0.91       183
           2       0.93      0.98      0.95       228
           3       0.99      0.99      0.99       226
           4       0.77      0.97      0.86       246
           5       0.97      0.99      0.98       231
           6       0.89      0.97      0.93       257
           7       0.96      0.96      0.96       222
           8       0.81      0.95      0.87       205
           9       0.91      0.96      0.94       254
          10       0.85      0.98      0.91       226
          11       0.98      0.99      0.99       263
          12       0.81      1.00      0.89       249
          13       0.90      0.96      0.93       216
          14       0.85      0.98      0.91       206
          15       0.74      0.96      0.

c:\Users\1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifi

In [48]:
# Берем векторизатор Bag of Words, который мы обучили на Train.
# используем только .transform()
print("Векторизация тестовой выборки...")
X_test_bag = vec_bag.transform(X_test['clean_instruction']).toarray()

# y_test_bin = mlb.transform(y_test['intent'])
print(f"Размер X_test: {X_test_bag.shape}")
print(f"Размер y_test: {y_test_bin.shape}")


# так как в цикле векторов ['BoW', 'TF-IDF'...] BoW шел первым:
model_bow = svc_list[0]

print("Предсказание на тестовых данных...")
y_pred_test = model_bow.predict(X_test_bag)

print("-" * 40)
print("ИТОГОВЫЕ МЕТРИКИ НА TEST (Bag of Words)")
print("-" * 40)

print(f"Micro F1:    {f1_score(y_test_bin, y_pred_test, average='micro'):.4f}")
print(f"Macro F1:    {f1_score(y_test_bin, y_pred_test, average='macro'):.4f}")
print(f"Weighted F1: {f1_score(y_test_bin, y_pred_test, average='weighted'):.4f}")
print("\nПолный отчет:")
print(classification_report(y_test_bin, y_pred_test))

Векторизация тестовой выборки...
Размер X_test: (6055, 3277)
Размер y_test: (6055, 27)
Предсказание на тестовых данных...
----------------------------------------
ИТОГОВЫЕ МЕТРИКИ НА TEST (Bag of Words)
----------------------------------------
Micro F1:    0.9349
Macro F1:    0.9365
Weighted F1: 0.9368

Полный отчет:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       246
           1       0.90      0.96      0.93       228
           2       0.94      0.94      0.94       210
           3       1.00      1.00      1.00       258
           4       0.80      0.96      0.87       217
           5       0.98      1.00      0.99       248
           6       0.86      0.95      0.90       262
           7       0.97      0.94      0.96       193
           8       0.89      0.96      0.92       240
           9       0.92      0.98      0.95       249
          10       0.85      0.98      0.91       202
          11       0.96      0.9

c:\Users\1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [49]:
from sklearn.metrics import accuracy_score, jaccard_score

# 1. Exact Match Ratio (Идеальное совпадение)
# Показывает % тикетов, где модель не ошиблась ни в одной запятой
exact_match = accuracy_score(y_test_bin, y_pred_test)

# 2. Jaccard Score (samples average)
# Показывает средний % перекрытия правильных и предсказанных тегов
jaccard = jaccard_score(y_test_bin, y_pred_test, average='samples')

# 3. Кастомная метрика: "Хотя бы один верный" (At Least One Match)
def at_least_one_accuracy(y_true, y_pred):
    # Умножаем матрицы: если есть совпадение 1*1, будет > 0
    matches = (y_true * y_pred).sum(axis=1)
    # Считаем долю строк, где сумма > 0
    return (matches > 0).mean()

at_least_one = at_least_one_accuracy(y_test_bin, y_pred_test)

print(f"--- ДОПОЛНИТЕЛЬНЫЕ МЕТРИКИ (BoW) ---")
print(f"Exact Match Ratio (Полная автоматизация): {exact_match:.4f}")
print(f"Jaccard Score (Степень похожести):        {jaccard:.4f}")
print(f"At Least One Match (Полезное действие):   {at_least_one:.4f}")

--- ДОПОЛНИТЕЛЬНЫЕ МЕТРИКИ (BoW) ---
Exact Match Ratio (Полная автоматизация): 0.8880
Jaccard Score (Степень похожести):        0.9319
At Least One Match (Полезное действие):   0.9782


In [50]:
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train_search = X_train_bag
y_train_search = y_train_bin

base_model = MultiOutputClassifier(LogisticRegression(random_state=42, max_iter=1000, solver='liblinear'))

param_grid = {
    'estimator__C': [0.1, 1, 10],
    'estimator__class_weight': [None, 'balanced'],
    'estimator__penalty': ['l1', 'l2']
}

grid_search = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    cv=3,
    scoring='f1_weighted',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_search, y_train_search)

print("\nЛучшие параметры:", grid_search.best_params_)
print("Лучший F1-score на кросс-валидации:", grid_search.best_score_)

best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_val_bag)

print("\nОтчет лучшей модели:")
print(classification_report(y_val_bin, y_pred_best))

Fitting 3 folds for each of 12 candidates, totalling 36 fits


: 

## Вопросы на подумать
- **подозрительно хороший скор** - кажется, я где-то жестоко неправ
- мультикласс - в нашем случае лучше onehotencode'ить каждую вариацию? или рассматривать комбинацию интентов как отдельный класс? unique-значения и играться порогами для моделей, чтобы было 1-2 метки предсказанных
- как бы чистить выбросы типо "question abobut, aaaaabout" и прочее? или их не надо чистить, считаем это нормальным шумом, который вполне реален? не теряет ли модель качество в этом шуме?
- ~~надо преобразование для трейна и валидации делать одной или разными векторайзерами? по логике кажется, что одним~~
- **пороги вероятностей для моделей**

### Мысли
- не сходится логрег и оч долго обучается - что-то странное
- точно нужно попробовать подбор гиперпараметров, с помощью GridSearchCV к примеру
- модель предсказала все нули??? стоит ли сделать , zero_division=0?

- думаем про стоп-слова и комменты повыше
- можно подумать над доп. метриками (взвешенный accuracy, кастомные)
- перебераем параметры, смотрим на метрики, мб графики
- попробовать ещё моделей
- пайплайн обучения, пайплайн применения - всё объединить

In [302]:
y_pred_list[0][np.sum(y_pred_list[0], axis=1) == 0][0], 
y_pred_list[0][np.sum(y_pred_list[0], axis=1) == 0].shape

(400, 27)

In [311]:
y_pred_list[0][np.sum(y_pred_list[0], axis=1) == 2].shape

(91, 27)

In [307]:
y_svc_pred_list[0][np.sum(y_svc_pred_list[0], axis=1) == 0][0], 
y_svc_pred_list[0][np.sum(y_svc_pred_list[0], axis=1) == 0].shape

(164, 27)

In [312]:
y_svc_pred_list[0][np.sum(y_svc_pred_list[0], axis=1) == 2].shape

(121, 27)

In [318]:
y_pred_list[0][np.sum(y_pred_list[0], axis=1) >= 2][0]

array([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

MultiOutputClassifier(estimator=LinearSVC(random_state=42))